# Data Mining: Klasifikasi Kelulusan (Fitur Upload)

Notebook ini menggunakan algoritma **Decision Tree** untuk memprediksi kelulusan.

**Fitur Baru:** Anda bisa mengupload file CSV secara langsung melalui tombol di bawah (khusus Google Colab).

## 1. Import Library & Upload Data

**Fungsi:**
- Jika di **Google Colab**: Muncul tombol "Choose Files" untuk upload.
- Jika di **Jupyter Lokal**: Otomatis mencari file `data_mahasiswa.csv` di folder yang sama.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io

# Logika untuk mendeteksi lingkungan (Colab vs Lokal)
try:
    from google.colab import files
    print("Mode Google Colab Terdeteksi. Silakan upload file CSV Anda:")
    
    # Memicu tombol upload
    uploaded = files.upload()
    
    # Mengambil file pertama yang diupload
    filename = next(iter(uploaded))
    df = pd.read_csv(io.BytesIO(uploaded[filename]))
    print(f"\nSukses! File '{filename}' berhasil dibaca.")

except ImportError:
    # Fallback untuk Jupyter Notebook Lokal (Offline)
    print("Mode Jupyter Lokal Terdeteksi.")
    print("Mencari file 'data_mahasiswa.csv' di folder ini...")
    try:
        df = pd.read_csv('data_mahasiswa.csv')
        print("Sukses! File ditemukan dan dibaca.")
    except FileNotFoundError:
        print("ERROR: File 'data_mahasiswa.csv' tidak ditemukan. Pastikan file ada di folder yang sama.")

# Menampilkan Preview Data
if 'df' in locals():
    print("\nPreview Data:")
    display(df.head())
else:
    print("Data belum dimuat. Silakan cek file Anda.")

## 2. Preprocessing Data

**Fungsi:**
1. Memisahkan atribut (IPK, Kehadiran, SKS) dan label (Lulus).
2. Mengubah label 'Lulus'/'Tidak Lulus' menjadi angka.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Cek apakah dataframe 'df' ada
if 'df' in locals():
    # 1. Split Fitur dan Target
    X = df[['IPK', 'Kehadiran_Persen', 'Jumlah_SKS']]
    y = df['Lulus']

    # 2. Encoding
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    
    # Menampilkan mapping agar kita tahu 0 itu apa, 1 itu apa
    mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    print(f"Mapping Label: {mapping}")

    # 3. Split Training & Testing
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
    print("Data siap untuk modeling.")
else:
    print("Jalankan cell sebelumnya terlebih dahulu untuk memuat data.")

## 3. Modeling & Evaluasi

**Fungsi:**
Melatih Decision Tree dan langsung melihat akurasinya.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

if 'X_train' in locals():
    # Modeling
    model = DecisionTreeClassifier(criterion='gini', random_state=42)
    model.fit(X_train, y_train)

    # Prediksi & Evaluasi
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)

    print(f"Akurasi Model: {acc * 100}%")
    print("\nDetail Laporan:")
    print(classification_report(y_test, y_pred, target_names=le.classes_))
else:
    print("Data training tidak ditemukan.")

## 4. Visualisasi & Prediksi Manual

**Fungsi:**
Melihat bentuk pohon keputusan dan mencoba input data manual.

In [ ]:
from sklearn.tree import plot_tree

if 'model' in locals():
    # Visualisasi
    plt.figure(figsize=(12,8))
    plot_tree(model, feature_names=['IPK', 'Kehadiran', 'SKS'], class_names=le.classes_, filled=True)
    plt.title("Pohon Keputusan Kelulusan")
    plt.show()

    # Tes Prediksi Manual
    # Format: [IPK, Kehadiran, SKS]
    input_data = np.array([[3.1, 80, 140]]) 
    hasil = model.predict(input_data)
    status = le.inverse_transform(hasil)[0]
    
    print(f"\nSimulasi Mahasiswa (IPK 3.1, Hadir 80%, SKS 140) -> Prediksi: {status}")